# Bootstrapped ROC-AUC

In [14]:
from typing import Tuple
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.metrics import roc_auc_score

def roc_auc_ci(
    classifier: ClassifierMixin,
    X: np.ndarray,
    y: np.ndarray,
    conf: float = 0.95,
    n_bootstraps: int = 10_000,
) -> Tuple[float, float]:
    '''
    Returns confidence bounds of ROC-AUC

    Input:
    classifier: classification model
    X: data
    y: target
    conf: confidence level
    n_bootstraps: count of bootstraps 

    Return:
    Tuple of confidence bounds
    '''

    def get_stratified_bootstrap_indices(bs):
        unique_strata, stratum_counts = np.unique(bs, return_counts=True)
        bs_index_list_stratified = np.concatenate([
            np.random.choice(np.where(bs == idx_stratum_var)[0], 
                             size=n_stratum_var,
                             replace=True)
            for idx_stratum_var, n_stratum_var in zip(unique_strata, stratum_counts)
        ])
        return bs_index_list_stratified
            
        auc_scores = []
        for _ in range(n_bootstraps):
            y = y.copy()
            indices = get_stratified_bootstrap_indices(y)
            y_true_bootstrap = y[indices]
            y_scores_bootstrap = classifier.predict_proba(X[indices])[..., 1]
            auc_scores.append(roc_auc_score(y_true_bootstrap, y_scores_bootstrap))

    lower_percentile = (1 - conf) / 2 * 100
    upper_percentile = 100 - lower_percentile
    lcb, ucb = np.percentile(auc_scores, [lower_percentile, upper_percentile])

    return (lcb, ucb)

In [2]:
# from typing import Tuple
# import numpy as np
# from sklearn.base import ClassifierMixin
# from sklearn.metrics import roc_auc_score

# def roc_auc_ci(
#     classifier: ClassifierMixin,
#     X: np.ndarray,
#     y: np.ndarray,
#     conf: float = 0.95,
#     n_bootstraps: int = 10_000,
# ) -> Tuple[float, float]:
#     '''
#     Returns confidence bounds of ROC-AUC

#     Input:
#     classifier: classication model
#     X: data
#     y: target
#     conf: confidence level
#     n_bootstraps: count of bootstraps 

#     Return:
#     Tuple of confidence bounds
#     '''
#     auc_scores = []
#     pos_class_indices = np.where(y == 1)[0]
#     neg_class_indices = np.where(y == 0)[0]
#     pos_coef = len(pos_class_indices) / len(y)
#     neg_coef = 1 - pos_coef

#     for _ in range(n_bootstraps):
#         indices_pos = np.random.choice(len(y[pos_class_indices]), size=round(len(y) * pos_coef))
#         indices_neg = np.random.choice(len(y[neg_class_indices]), size=round(len(y) * neg_coef))
#         indices = np.hstack((indices_neg, indices_pos))
#         y_true_bootstrap = y[indices]

#         y_scores_bootstrap = classifier.predict(X[indices])

#         if len(np.unique(y_true_bootstrap)) > 1:
#             auc_scores.append(roc_auc_score(y_true_bootstrap, y_scores_bootstrap))

#     if len(auc_scores) == 0:
#         raise ValueError("No valid ROC AUC scores calculated. Ensure both classes are present in the data.")

    # lower_percentile = (1 - conf) / 2 * 100
    # upper_percentile = 100 - lower_percentile
    # lcb = np.percentile(auc_scores, lower_percentile)
    # ucb = np.percentile(auc_scores, upper_percentile)
    
#     return (lcb, ucb)


In [25]:
def roc_auc_ci(
    classifier: ClassifierMixin,
    X: np.ndarray,
    y: np.ndarray,
    conf: float = 0.95,
    n_bootstraps: int = 10_000,
) -> Tuple[float, float]:
    auc_scores = np.zeros(n_bootstraps)

    def get_stratified_bootstrap_indices(bs):
        unique_strata, stratum_counts = np.unique(bs, return_counts=True)
        bs_index_list_stratified = np.concatenate([
            np.random.choice(np.where(bs == idx_stratum_var)[0], 
                             size=n_stratum_var,
                             replace=True)
            for idx_stratum_var, n_stratum_var in zip(unique_strata, stratum_counts)
        ])
        return bs_index_list_stratified
    
    for i in range(n_bootstraps):
        y = y.copy()
        indices = get_stratified_bootstrap_indices(y)
        y_true_bootstrap = y[indices]
        y_scores_bootstrap = classifier.predict_proba(X[indices])[..., 1]
        auc_scores[i] = roc_auc_score(y_true_bootstrap, y_scores_bootstrap)

    lower_percentile = (1 - conf) / 2 * 100
    upper_percentile = 100 - lower_percentile
    lcb, ucb = np.percentile(auc_scores, [lower_percentile, upper_percentile])
    
    return lcb, ucb


In [1]:
from typing import Tuple
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.metrics import roc_auc_score


def roc_auc_ci(
    classifier: ClassifierMixin,
    X: np.ndarray,
    y: np.ndarray,
    conf: float = 0.95,
    n_bootstraps: int = 10_000,
) -> Tuple[float, float]:
    '''
    Returns confidence bounds of ROC-AUC

    Input:
    classifier: classification model
    X: data
    y: target
    conf: confidence level
    n_bootstraps: count of bootstraps 

    Return:
    Tuple of confidence bounds
    '''

    auc_scores = np.zeros(n_bootstraps)

    def get_stratified_bootstrap_indices(bs):
        unique_strata, stratum_counts = np.unique(bs, return_counts=True)
        bs_index_list_stratified = np.concatenate([
            np.random.choice(np.where(bs == idx_stratum_var)[0], 
                             size=n_stratum_var,
                             replace=True)
            for idx_stratum_var, n_stratum_var in zip(unique_strata, stratum_counts)
        ])
        return bs_index_list_stratified
    predicts = classifier.predict_proba(X)[..., 1]
    
    for i in range(n_bootstraps):
        y = y.copy()
        indices = get_stratified_bootstrap_indices(y)
        y_true_bootstrap = y[indices]
        y_scores_bootstrap = predicts[indices]
        auc_scores[i] = roc_auc_score(y_true_bootstrap, y_scores_bootstrap)

    lower_percentile = (1 - conf) / 2 * 100
    upper_percentile = 100 - lower_percentile
    lcb, ucb = np.percentile(auc_scores, [lower_percentile, upper_percentile])
    
    return lcb, ucb


In [2]:
from time
from typing import Tuple

import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression


classifier = LogisticRegression()
X = np.array([np.random.uniform(0, 6, 10) for _ in range(10000)])
y = np.random.binomial(1, 0.1, 10000)
X_test = np.array([np.random.uniform(1, 3, 10) for _ in range(100)])
y_test = np.random.binomial(1, 0.3, 100)
classifier.fit(X, y)

roc_auc_ci(classifier, X_test, y_test, n_bootstraps=100000, conf=0.95)

KeyboardInterrupt: 

In [149]:
# def roc_auc_ci(
#     classisfier: ClassifierMixin,
#     X: np.array,
#     y: np.array,
#     conf: float = 0.95,
#     n_bootstraps: int = 10_000,
# ) -> Tuple[float, float]:
#     '''
#     Returns confidence bounds of ROC-AUC

#     Input:
#     classifier: classication model
#     X: data
#     y: target
#     conf: confidence level
#     n_bootstraps: count of bootstraps 

#     Return:
#     Tuple of confidence bounds
#     '''

#     auc_scores = []
#     strat_split = StratifiedKFold(n_splits=n_bootstraps, shuffle=True, )
    
#     for _, test_index in strat_split.split(X, y):
#         X_test = X[test_index]
#         y_test = y[test_index]

#         y_scores_bootstrap = classifier.predict(X_test)
#         auc_scores.append(roc_auc_score(y_test, y_scores_bootstrap))

#     lower_percentle = (1 - conf) / 2 * 100
#     upperr_percentle = 100 - lower_percentle

#     return (np.percentile(auc_scores, lower_percentle), np.percentile(auc_scores, upperr_percentle))

In [60]:
from typing import Tuple
from sklearn.metrics import precision_recall_curve, auc, roc_curve
import matplotlib.pyplot as plt
import numpy as np




def pr_threshold(y_true, y_prob, min_precision):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob, )
    precision = precision[:-1]
    recall = recall[:-1]
    valid_thresholds = thresholds[precision >= min_precision]
    
    if len(valid_thresholds) > 0:
        max_recall_index = np.argmax(recall[precision >= min_precision])
        threshold_proba = valid_thresholds[max_recall_index]
        max_recall = recall[precision >= min_precision][max_recall_index]
    else:
        threshold_proba = max_recall = 0.0
    
    return threshold_proba, max_recall


def sr_threshold(
    y_true: np.ndarray,
    y_prob: np.ndarray,
    min_specificity: float,
) -> Tuple[float, float]:
    """Returns threshold and recall (from Specificity-Recall Curve) using optimization"""
    fpr, recall, thresholds = roc_curve(y_true, y_prob)

    # Calculate specificity in a vectorized way
    specificity = 1 - fpr
    valid_thresholds = thresholds[specificity >= min_specificity]


    if len(valid_thresholds) > 0:
        max_recall_index = np.argmax(recall[specificity >= min_specificity])
        threshold_proba = valid_thresholds[max_recall_index]
        max_recall = recall[specificity >= min_specificity][max_recall_index]
    else:
        threshold_proba = max_recall = 0.0
    threshold_proba = max(0, min(threshold_proba, 1))
    max_recall = max(0, min(max_recall, 1))

    return threshold_proba, max_recall




def bootstrap_auc(y_true, y_prob, conf, n_bootstrap):
    np.random.seed(42)
    auc_values = []
    
    for _ in range(n_bootstrap):
        indices = np.random.choice(len(y_true), len(y_true), replace=True)
        y_true_bootstrap = y_true[indices]
        y_prob_bootstrap = y_prob[indices]
        precision, recall, _ = precision_recall_curve(y_true_bootstrap, y_prob_bootstrap)
        auc_values.append(auc(recall, precision))
    
    auc_values.sort()
    lower_percentile, upper_percentile = np.percentile(auc_values, [(1 - conf) / 2 * 100, 100 - (1 - conf) / 2 * 100])
    return lower_percentile, upper_percentile


def pr_curve(y_true, y_prob, conf=0.95, n_bootstrap=10_000):
    precision, recall, thresholds = compute_precision_recall_threshold(y_true, y_prob, conf)
    precision_lcb, precision_ucb = bootstrap_auc(y_true, y_prob, conf, n_bootstrap)
    return recall, precision, precision_lcb, precision_ucb


def sr_curve(y_true, y_prob, conf=0.95, n_bootstrap=10_000):
    precision, recall, thresholds = compute_precision_recall_threshold(y_true, y_prob, conf)
    specificity = 1 - np.array([np.sum((y_prob >= threshold) & (y_true == 0)) / np.sum(y_true == 0) for threshold in thresholds])
    specificity_lcb, specificity_ucb = bootstrap_auc(y_true, 1 - y_prob, conf, n_bootstrap)
    return recall, specificity, specificity_lcb, specificity_ucb


In [2]:
N = 10**5
y_true =np.array([np.random.choice(np.random.binomial(1, 1, 100)) for _ in range(N)])
y_prob =np.array([np.random.choice(np.random.normal(0, 1, 100)) for _ in range(N)])
min_precision = 0.2
min_specificity = 0.4
conf = 0.95
n_bootstrap = 10**3

In [17]:
N = 10**7
y_true = np.array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1])
y_prob =np.array([0.8, 0.7, 0.1, 0.2, 0.15, 0.1, 0., 0.2, 0.6, .2, .3, .9, 0.1, .8])
min_precision = 0.8
min_specificity = 0.4
conf = 0.8
n_bootstrap = 10**3

In [65]:
def sr_threshold(
    y_true: np.ndarray,
    y_prob: np.ndarray,
    min_specificity: float,
) -> Tuple[float, float]:
    """Returns threshold and recall (from Specificity-Recall Curve) using optimization"""
    fpr, recall, thresholds = roc_curve(y_true, y_prob)

    specificity = 1 - fpr
    valid_indices = np.where(specificity >= min_specificity)

    max_recall_index = np.argmax(recall[valid_indices])
    max_recall = recall[max_recall_index]
    threshold_proba = thresholds[max_recall_index]
    
    threshold_proba = max(0, min(1, threshold_proba))
    return threshold_proba, max_recall



In [67]:
sr_threshold(y_true, y_prob, min_specificity)

(0.2, 0.7777777777777778)

In [22]:
pr_curve(y_true, y_prob, conf, n_bootstrap)

(array([1.00000000e+00, 9.99988908e-01, 9.99977815e-01, ...,
        2.21845086e-05, 1.10922543e-05, 0.00000000e+00]),
 array([0.90153   , 0.90152902, 0.90152803, ..., 1.        , 1.        ,
        1.        ]),
 0.8992355474895963,
 0.9045811472722523)

In [23]:
sr_curve(y_true, y_prob, conf, n_bootstrap)

(array([1.00000000e+00, 9.99988908e-01, 9.99977815e-01, ...,
        2.21845086e-05, 1.10922543e-05, 0.00000000e+00]),
 array([0., 0., 0., ..., 1., 1., 1.]),
 0.8991005755904459,
 0.9042566040756608)